In [65]:
# importing necessary packages 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os 
from functools import reduce
from sklearn.preprocessing import StandardScaler

#survey_current_df = pd.read_csv('/Users/rfernex/Documents/Education/SciencesPo/Courses/CSS/Projects/Project BoJ (Ongoing)/Data/processed/survey_current_situation_df.csv')
#survey_future_df = pd.read_csv('/Users/rfernex/Documents/Education/SciencesPo/Courses/CSS/Projects/Project BoJ (Ongoing)/Data/processed/survey_future_situation_df.csv')
#survey_past_df = pd.read_csv('/Users/rfernex/Documents/Education/SciencesPo/Courses/CSS/Projects/Project BoJ (Ongoing)/Data/processed/survey_past_situation_df.csv')
#survey_perception_df = pd.read_csv('/Users/rfernex/Documents/Education/SciencesPo/Courses/CSS/Projects/Project BoJ (Ongoing)/Data/processed/survey_perception_rates_df.csv')
#Statements_boj_df = pd.read_csv('/Users/rfernex/Documents/Education/SciencesPo/Courses/CSS/Projects/Project BoJ (Ongoing)/Data/processed/Statements_boj_df.csv')



In [27]:
# removes NA columns and creates a copy
survey_current_df_clean = survey_current_df[~survey_current_df.astype(str).isin(['-']).any(axis=1)].copy()
# Converts data to numeric
for col in survey_current_df_clean.columns[1:6]:
    survey_current_df_clean[col] = pd.to_numeric(survey_current_df_clean[col], errors='coerce')
    
display(survey_current_df_clean.info())

<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 2 to 74
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   month_year            73 non-null     object 
 1   Favorable             73 non-null     float64
 2   Somewhat.favorable    73 non-null     float64
 3   Difficult.to.say      73 non-null     float64
 4   Somewhat.unfavorable  73 non-null     float64
 5   Unfavorable           73 non-null     float64
dtypes: float64(5), object(1)
memory usage: 4.0+ KB


None

In [47]:
display(survey_perception_df.head())

,month_year,Too.low,Appropriate,Too.high
0,2006-06,64.8,22.3,9.9
1,2006-09,64.2,25.4,7.7
2,2006-12,63.5,23.0,10.9
3,2007-03,58.2,28.4,10.5
4,2007-06,60.7,23.2,11.8


In [51]:
# Building weighted sentiment scores                      

#current sentiment
weights_array_current = np.array([2, 1, 0, -1, -2])
survey_current_df_clean["wgt_current"] = np.dot(survey_current_df_clean[["Favorable", "Somewhat.favorable", "Difficult.to.say", "Somewhat.unfavorable", "Unfavorable"]], weights_array_current)

#past sentiment
weights_array_future = np.array([1, 0, -1])
survey_future_df["wgt_future"] = np.dot(survey_future_df[["Will.improve", "Will.remain.the.same", "Will.worsen"]], weights_array_future)

#future sentiment
weights_array_past = np.array([1, 0, -1])
survey_past_df["wgt_past"] = np.dot(survey_past_df[["Have.improved", "Have.remained.the.same", "Have.worsened"]], weights_array_past)


#perception sentiment
weights_array_perception = np.array([-1, 0, 1])
survey_perception_df["wgt_perception"] = np.dot(survey_perception_df[["Too.low", "Appropriate", "Too.high"]], weights_array_perception)

display(survey_current_df_clean.head())
display(survey_future_df.head())
display(survey_past_df.head())
display(survey_perception_df.head())



,month_year,Favorable,Somewhat.favorable,Difficult.to.say,Somewhat.unfavorable,Unfavorable,wgt_current
2,2006-12,0.6,13.2,40.7,35.6,9.5,-40.2
3,2007-03,0.6,12.4,40.8,35.9,10.0,-42.3
4,2007-06,1.0,14.0,39.8,35.2,9.9,-39.0
5,2007-09,0.6,8.9,35.6,41.6,13.0,-57.5
6,2007-12,0.2,6.6,28.9,47.4,16.7,-73.8


,month_year,Will.improve,Will.remain.the.same,Will.worsen,wgt_future
0,2006-06,16.5,62.3,20.3,-3.8
1,2006-09,13.8,65.7,19.7,-5.9
2,2006-12,10.3,67.5,21.7,-11.4
3,2007-03,12.3,67.0,20.0,-7.7
4,2007-06,10.2,64.3,24.7,-14.5


,month_year,Have.improved,Have.remained.the.same,Have.worsened,wgt_past
0,2006-06,15.1,62.4,21.8,-6.7
1,2006-09,11.9,65.0,22.9,-11.0
2,2006-12,11.1,65.3,23.3,-12.2
3,2007-03,11.1,65.3,23.2,-12.1
4,2007-06,11.6,64.1,23.6,-12.0


,month_year,Too.low,Appropriate,Too.high,wgt_perception
0,2006-06,64.8,22.3,9.9,-54.9
1,2006-09,64.2,25.4,7.7,-56.5
2,2006-12,63.5,23.0,10.9,-52.6
3,2007-03,58.2,28.4,10.5,-47.7
4,2007-06,60.7,23.2,11.8,-48.9


In [59]:

survey_results = [survey_current_df_clean[["month_year","wgt_current"]], survey_future_df[["month_year","wgt_future"]], survey_past_df[["month_year","wgt_past"]], survey_perception_df[["month_year","wgt_perception"]]]
merged_survey_results = reduce(lambda left, right: pd.merge(left, right, on='month_year', how='outer'), survey_results).iloc[2:]
display(merged_survey_results.head())
display(merged_survey_results.info())


,month_year,wgt_current,wgt_future,wgt_past,wgt_perception
2,2006-12,-40.2,-11.4,-12.2,-52.6
3,2007-03,-42.3,-7.7,-12.1,-47.7
4,2007-06,-39.0,-14.5,-12.0,-48.9
5,2007-09,-57.5,-22.9,-27.4,-44.7
6,2007-12,-73.8,-41.4,-40.7,-46.3


In [71]:

# Create a copy of the dataframe
survey_results_scaled = merged_survey_results.copy()

# Select all columns except month_year for scaling
columns_to_scale = [col for col in merged_survey_results.columns if col != 'month_year']

# Apply scaling only to selected columns
scaler = StandardScaler()
survey_results_scaled[columns_to_scale] = scaler.fit_transform(merged_survey_results[columns_to_scale])

display(survey_results_scaled.head())


,month_year,wgt_current,wgt_future,wgt_past,wgt_perception
2,2006-12,1.028188,1.006990,1.212883,-1.724966
3,2007-03,0.967818,1.304342,1.217315,-1.141826
4,2007-06,1.062686,0.757857,1.221747,-1.284636
5,2007-09,0.530851,0.082787,0.539235,-0.784802
6,2007-12,0.062261,-1.403973,-0.050208,-0.975215


In [69]:
display(merged_survey_results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 2 to 74
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   month_year      73 non-null     object 
 1   wgt_current     73 non-null     float64
 2   wgt_future      73 non-null     float64
 3   wgt_past        73 non-null     float64
 4   wgt_perception  73 non-null     float64
dtypes: float64(4), object(1)
memory usage: 3.0+ KB


None